In [ ]:
import sys
sys.path.append('../')

import utils.evaluation as ueva
import pipeline.sql as plsql

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from plotnine import *
theme_set(theme_bw())

In [ ]:
engine = plsql.create_engine('../config.yaml')
role = 'direccion_trabajo_inspections_write'

In [ ]:
# Model ID graphs
# This is in the pipeline
#pleva.model_id_graphs(141, role, engine, "")

In [ ]:
model_group_list = [str(x) for x in range(139, 400)]
cutoff_list = [str(x) for x in [2000, 3000, 4000]]
model_group_list[0]

In [ ]:
model_group_list[0]

In [ ]:
# Model Group graphs
ueva.model_group_id_graphs(model_group_list, cutoff_list,  role, engine)

In [ ]:
#select model_group_id from results.model_groups order by model_group_id desc limit 15;

In [ ]:
pleva.comp_modelid_jaccard_scores(353, 354, 2000, role, engine)

In [ ]:
qry = """
SET ROLE {};
SELECT 
    model_group_id,
    model_type,
    metric_name,
    null_par,
    AVG(value) as avg, 
    stddev(value) as sd
FROM results.view_modelgroup 
WHERE cutoff = '2000' AND 
    metric_name = 'precision' AND
    null_par = 'all'
GROUP BY model_group_id, metric_name, null_par, model_type
ORDER BY avg desc, sd asc;
""".format(role)
df = pd.read_sql_query(qry, engine)

In [ ]:
df[19:29]

In [ ]:
group_ins = '282'
group_vio = '540'

qry = """SET ROLE {}; 
    SELECT * 
    FROM 
        (SELECT entity_id, 
            score as score_ins, 
            label_value as label_value_ins, 
            as_of_date as as_of_date_ins
        FROM results.predictions
        WHERE model_id = '{group_ins}') i
    INNER JOIN
        (SELECT entity_id, 
            score AS score_vio, 
            CASE WHEN label_value IS NULL THEN 'not inspected' 
                ELSE cast(label_value as VARCHAR) 
                END AS label_value_vio, 
            as_of_date AS as_of_date_vio
        FROM results.predictions
        WHERE model_id = '{group_vio}') v
    ON i.entity_id = v.entity_id;
    """.format(role, 
               group_ins = group_ins, 
               group_vio = group_vio)
df = pd.read_sql_query(qry, engine)
## order the features
order_features = df.label_value_vio.unique().tolist()
df['violation'] = df.label_value_vio.astype('category', 
                                ordered = True,
                                categories = order_features)
(ggplot(df, aes(x = 'score_ins', 
                y = 'score_vio')) + 
    geom_point(alpha = .1, size = 2) +
    #scale_color_manual(values=('gray', 'salmon', 'green')) + 
    xlab('Inspection score') + 
    ylab('Violation score'))

In [ ]:
df.head()

In [ ]:
#df['label_value_vio'][pd.isna(df['label_value_vio'])] = 'Never inspected'
#df_rep = df.replace([None], ['Never inspected'])


In [ ]:
## create categorical version for graph


In [ ]:
#groupid_metricat_eval_time(role, engine, group_ins, group_vio metric_name = 'precision', cutoff = 2000)
#model_id, as_of_date, entity_id, score,
#        model_group_id, s.model_type, s.label
#model_group_id, model_id, a.model_type, a.label
group_ins = '282'
group_vio = '196'

def comp_modelid_inspections_violations(model_id_ins, model_group_id_vio):

qry = """set role {}; 
    select *
    from results.predictions 
	left join 
		(select *
		from results.models 
	   	left join 
			(select model_group_id, model_type, label
	    	from results.model_groups) a
	  	using (model_group_id)) s
	using (model_id) 
   	where model_id in ({model_id});
    """.format(role, model_group_id = ', '.join([group_ins, group_vio]))
qry
df = pd.read_sql_query(qry, engine)

tab = df.loc[:, ["entity_id", "label", "score"]].reset_index()
tabc = tab.pivot_table(index=['entity_id'], columns='label', values='score').reset_index()

(ggplot(tabc, aes(x = 'inspected', y = 'infractor')) + 
    geom_point(alpha = .5) +
    xlab('Inspection score\n'.format(model_id_ins)) + 
    ylab('Violation score'.format(model_id_vio)))



In [ ]:
df.head()

In [ ]:
#tab_label = df[tab['label'] == 'infractor'].reset_index()
#tab_label['violation label'] = tab_label.label_value
#tabc.merge(tab_label[['entity_id', 'violation label']], left_on = 'entity_id', right_on = 'entity_id')


In [ ]:
qry = """
SET ROLE direccion_trabajo_inspections_write;
select random_feature, infractor
from staging.train_monthly_split_20120201_20120301;
"""
df = plsql.query(qry, engine)

In [ ]:
#(qplot(x = df.infractor, y = df.random_feature))
(ggplot(df, aes(x = 'infractor', y = 'random_feature')) + 
    geom_boxplot())